1\. Write a function that converts number representation (bin<->dec<->hex)

In [83]:
import string

def change_rep(x):
    binary = None
    hexa = None
    inter = None
    
    if isinstance(x, str):
        for i in x:
            if i in '01':
                binary=True
            elif i in string.hexdigits:
                hexa=True
                binary=False
            else:
                binary=False
                hexa=False
    elif isinstance(x, int):
            inter = True
    else:
        inter = False
        print('value not acceptable')
            
    if binary == True:
        x_dec = int(x, 2)
        x_hex = hex(x_dec)
        print('x is binary, x_dec =', x_dec, " x_hex = ", x_hex)
    elif hexa == True:
        x_dec = int(x, 16)
        x_bin = bin(x_dec)
        print('x is hexadeciaml, x_bin =', x_bin, " x_dec = ", x_dec)
    elif inter == True:
        x_bin = bin(x)
        x_hex = hex(x)
        print('x is decimal, x_bin =', x_bin, " x_hex = ", x_hex)
    else: 
        print('write a binary, deciaml or integer')

change_rep('')

x is hexadeciaml, x_bin = 0b110100011  x_dec =  419


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [126]:
def s32_to_float(x):
    binary = None
    if isinstance(x, str) and len(x)==32:
        for i in x:
            if i in '01':
                binary=True
            else:
                binary=False
                print('no binary')
    else:
        print('no 32 bit')
        
    if binary==True:
        #sign
        s = x[0]
        #exponent
        e = x[1:9]
        ef = int(e, 2)
        #mantissa
        m = x[9:]
        mf=0
        for i in range(0, len(m)):
            mf = mf + int(m[i])*2**(-i-1)
        #final
        f = (-1)**int(s) * (1 + mf) * pow(2, ef-127)
        print(f)
    else:
        print('write a correct string')
        
s32_to_float('11000000101100000000000000000000')

-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [4]:
def over(x):
    stat=True
    while stat==True:
        try:
            x = x*2
        except:
            stat=False
            print(x)
            
over(1)

...


In [2]:
def under(x):
    stat=True
    while stat==True:
        try:
            x = x/2
        except:
            stat=False
            print(x)
            
under(1)

0.0


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [3]:
for e in [14,15,16]: print (7+1.0*10**-e)

7.00000000000001
7.000000000000001
7.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [1]:
import math

#a
def root2(a, b, c):
    x1 = (-b + math.sqrt(b**2-4*a*c))/(2*a)
    x2 = (-b - math.sqrt(b**2-4*a*c))/(2*a)
    print(x1, x2)
    
#b    
def root21(a, b, c):
    x1 = (4*a*c)/((-b-math.sqrt(b**2-4*a*c))*2*a)
    x2 = (4*a*c)/((-b+math.sqrt(b**2-4*a*c))*2*a)
    print(x1, x2)
    
#c
def root22(a, b, c):
    x1 = (-b + math.sqrt(b**2-4*a*c))/(2*a)
    x2 = c/(a*x1)
    print(x1, x2)
    
root2(0.001, 1000, 0.001)
root21(0.001, 1000, 0.001)
root22(0.001, 1000, 0.001)

-9.999894245993346e-07 -999999.999999
-1.0000000000009999e-06 -1000010.5755125057
-9.999894245993346e-07 -1000010.5755125057


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [14]:
def f(x):
    return x*(x-1)

d= 10**(-12)
def dev(f, d):
    return (f(1+d)-f(1))/d

dev(f, d)

1.0000889005833413

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [2]:
import math
import numpy as np

def f(x):
    return math.sqrt(1-x**2)

def riem(a, b, N):
    x = np.linspace(a, b, N)
    I = 0
    for i in x:
        I = I + f(i)
    return (I*2)/N

In [34]:
%timeit riem(-1, 1, 1000000)

1 loop, best of 3: 709 ms per loop


In [4]:
riem(-1, 1, 1000000)

1.5707947543354452